In [1]:
# Basic Imports
import numpy as np
import h5py as h5
#from sklearn.externals import joblib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.cross_validation import KFold
import pickle
import re

In [2]:
#datasets

#input data
train_val_file = '/projects/nikhil/ADNI_prediction/input_datasets/cli_ct_seg_fused_train_plus_val.pkl'
test_file = '/projects/francisco/data/ADNI/cli_ct_seg_fused_test.pkl'

#k-fold indices (from a saved file)
kf_file = "/projects/nikhil/ADNI_prediction/input_datasets/cli_ct_train_valid_KFold_idx.pkl"

#save dir for trained model 
CV_model_dir = '/projects/nikhil/ADNI_prediction/models/CV_pkls/'

In [3]:
#Grab CV data with specific feature columes (independent vars) and specific clinical scale (dependent var)
def load_CV_data(in_file, kf_file, feature_cols, clinical_scale):

    data = pd.read_pickle(in_file)
    data_trunc = data[clinical_scale + feature_cols]
    # remove nans
    data_trunc = data_trunc[np.isfinite(data_trunc[clinical_scale[0]])]
    X = np.asarray(data_trunc[feature_cols],dtype=float)
    y = np.asarray(data_trunc[clinical_scale[0]],dtype=float)
    
    kf = pickle.load( open(kf_file, "rb" ) )
    X_train = []
    X_valid = []
    y_train = []
    y_valid = []
    for train, valid in kf:        
        X_train.append(X[train])
        X_valid.append(X[valid])
        y_train.append(y[train])
        y_valid.append(y[valid])
    
    print valid
    # Return train and validation lists comprising all folds as well as unsplit data
    return {'X_train':X_train,'X_valid':X_valid,'y_train':y_train,'y_valid':y_valid,'X':X,'y':y}

#Load test data
def load_test_data(in_file, feature_cols, clinical_scale):

    data = pd.read_pickle(in_file)
    data_trunc = data[clinical_scale + feature_cols]
    # remove nans
    data_trunc = data_trunc[np.isfinite(data_trunc[clinical_scale[0]])]
    X = np.asarray(data_trunc[feature_cols],dtype=float)
    y = np.asarray(data_trunc[clinical_scale[0]],dtype=float)
    return {'X':X, 'y':y}

def innerCVLoop(model_clf,hyper_params,fold_X, fold_y,save_model,save_model_path):
    clf = grid_search.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0)
    clf.fit(fold_X, fold_y)
    #Save classifier
    if save_model:
        save_model(clf,save_model_path)
        
    return clf

def save_classifier(clf,save_model_path):
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d-%H-%M-%S')
    save_model_filename = save_model_path + '_' + st + '.pkl'
        
    f = open(save_model_filename, 'wb')
    pickle.dump(clf, f)
    f.close()

In [4]:
# Grab specific columns as features from the original table
data = pd.read_pickle(train_val_file)
data_cols = data.columns
regex=re.compile(".*(CT_).*")
CT_cols = [m.group(0) for l in data_cols for m in [regex.search(l)] if m] 

regex=re.compile(".*(_HC_)(\d+)")
HC_cols = [m.group(0) for l in data_cols for m in [regex.search(l)] if m] 

#feature_cols = ['L_HC_VOL','R_HC_VOL'] + CT_cols
feature_cols = HC_cols + CT_cols
clinical_scale = ['ADAS13']

cv_data = load_CV_data(train_val_file,kf_file, feature_cols, clinical_scale)
test_data = load_test_data(test_file, feature_cols, clinical_scale)

len(data_cols), len(CT_cols), len(HC_cols), 11427+10519

[  8   9  11  14  26  27  48  52  53  55  73  75  83  88  91  93 100 101
 105 108 109 122 148 149 162 167 169 173 176 177 189 190 200 211 226 241
 349 353 366 373 392 400 420 433 442 451 492 496 499 506 541 543 556 558
 566 575 578 579]


(22029, 74, 21946, 21946)

In [58]:
# Use this data for computing subject wise performance during outerloop cross-validation + held-out testset
fused_data_dir = '/projects/nikhil/ADNI_prediction/input_datasets/'
CV_fused_file = 'HC_CT_inflated_CV_OuterFolds_valid_partition_fused.h5'
test_fused_file = 'HC_CT_inflated_CV_OuterFolds_test_partition_fused.h5'

CV = True
L_HC_offset=11427
R_HC_offset=10519

if CV:
    cv_data_valid_X = cv_data['X_valid']
    cv_data_valid_y = cv_data['y_valid']

    CV_fused_data = h5.File(fused_data_dir + CV_fused_file,'a')
    for f in np.arange(10):
        all_data = cv_data_valid_X[f]
        CV_fused_data.create_dataset('Fold_{}_X_L_HC'.format(f+1),data=all_data[:,:L_HC_offset])
        CV_fused_data.create_dataset('Fold_{}_X_R_HC'.format(f+1),data=all_data[:,L_HC_offset:L_HC_offset+R_HC_offset])
        CV_fused_data.create_dataset('Fold_{}_X_CT'.format(f+1),data=all_data[:,L_HC_offset+R_HC_offset:])
        CV_fused_data.create_dataset('Fold_{}_y'.format(f+1),data=cv_data_valid_y[f])
    
    CV_fused_data.close()

else:
    test_fused_data = h5.File(fused_data_dir + test_fused_file,'a')
    all_data = test_data['X']
    test_fused_data.create_dataset('test_X_L_HC'.format(f+1),data=all_data[:,:L_HC_offset])
    test_fused_data.create_dataset('test_X_R_HC'.format(f+1),data=all_data[:,L_HC_offset:L_HC_offset+R_HC_offset])
    test_fused_data.create_dataset('test_X_CT'.format(f+1),data=all_data[:,L_HC_offset+R_HC_offset:])    
    test_fused_data.create_dataset('test_y',data=test_data['y'])
    test_fused_data.close()


In [30]:
# K-fold validations (nested)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import grid_search
import datetime
import time
import collections
from scipy.stats import mode
from sklearn.metrics import mean_squared_error as mse

model_list = ['LR', 'LR_L1', 'SVR', 'RFR']
model_choice_id = 3
model_choice = model_list[model_choice_id]

if model_choice == 'LR':
    model_clf = LinearRegression()   
    inner_loop = False #only needed to optimize hyper-params
    feat_imp = False
    saved_model_name = 'LR_2015-10-13-16-38-28.pkl'
    
elif model_choice == 'LR_L1':
    model_clf = Lasso()
    hyper_params = {'alpha':[0.2, 0.1, 0.05, 0.01]} 
    inner_loop = True #only needed to optimize hyper-params
    feat_imp = False
    if clinical_scale[0] == 'ADAS13':
        saved_model_name = 'LR_L1_2015-10-13-16-47-35.pkl'
    else:
        saved_model_name = 'LR_L1_MMSE_2015-10-16-16-27-29.pkl'
    
elif model_choice == 'SVR':
    model_clf = SVR()
    hyper_params = {'kernel':('linear', 'rbf'), 'C':[1,2.5,5,7.5,10]}
    inner_loop = True #only needed to optimize hyper-params
    feat_imp = False
    if clinical_scale[0] == 'ADAS13':
        saved_model_name = 'SVR_2015-10-14-13-40-45.pkl'
    else:
        saved_model_name = 'SVR_MMSE_2015-10-16-18-02-11.pkl'
    
elif model_choice == 'RFR':
    model_clf = RandomForestRegressor()
    hyper_params = {'n_estimators':[50,100,200,500],'min_samples_split':[1,2,4,8]}
    inner_loop = True #only needed to optimize hyper-params
    feat_imp = True
    if clinical_scale[0] == 'ADAS13':
        saved_model_name = 'RFR_2015-10-13-15-15-11.pkl'
    else:
        saved_model_name = 'RFR_MMSE_2015-10-16-16-22-42.pkl'
else:
    print "Unknown model choice"

In [31]:
load_model_path = CV_model_dir + saved_model_name

# train a new classifer? (if false then load a single pretrained classifier based on most frequent hyperparams found previously
# This will NOT load N different classifiers each for outer-CV fold  
train_clf = False

cv_X_train = cv_data['X_train']
cv_y_train = cv_data['y_train']
cv_X_valid = cv_data['X_valid']
cv_y_valid = cv_data['y_valid']

no_of_folds = len(cv_X_train)

CV_R2_train=[] #R2 score for each outer fold on train set
CV_R2_valid=[] #R2 score for each outer fold on validation set

CV_MSE_train=[] #MSE for each outer fold on train set
CV_MSE_valid=[] #MSE for each outer fold on validation set

save_model = False #do you want to save all classifiers per each fold? 
if train_clf:
    # outer CV loop
    hp_dict = collections.defaultdict(list) #store best hyper-parameters for each fold
    for i in np.arange(no_of_folds):
        fold_X = cv_X_train[i]
        fold_y = cv_y_train[i]
        
        if inner_loop: 
            save_model_path_fold = save_model_path + '_fold_' + str(i)
            clf = innerCVLoop(model_clf,hyper_params,fold_X, fold_y,save_model,save_model_path_fold)
            for hp in hyper_params:
                hp_dict[hp].append(clf.best_estimator_.get_params()[hp])

        else:
            clf = model_clf
            clf.fit(fold_X,fold_y)
        
        #CV_score
        CV_R2_train.append(clf.score(cv_X_train[i],cv_y_train[i]))
        CV_R2_valid.append(clf.score(cv_X_valid[i],cv_y_valid[i]))
        
        CV_MSE_train.append(mse(clf.predict(cv_X_train[i]),cv_y_train[i]))
        CV_MSE_valid.append(mse(clf.predict(cv_X_valid[i]),cv_y_valid[i]))
           
    #retrain model on the entire train + valid set with most frequent hyper-params during cross-val
    if inner_loop:
        hp_mode = {}
        for hp in hyper_params:
            hp_mode[hp] = mode(hp_dict[hp])[0][0]
            
        print 'most frequent hp:' + str(hp_mode)
    
else: 
    #Grabs the best classifer as a result of N-fold nested CV along with the MSE and R2 stats of the outerloop
    print "Loading previously saved model: "
    f = open(load_model_path)
    result = pickle.load(f)
    test_clf = result['best_clf']
    CV_R2_valid = result['CV_R2']
    CV_MSE_valid = result['CV_MSE']
    f.close()
    
print 'CV R2 (mean, std_err): ' + '{:04.2f}, {:04.2f}'.format(np.mean(CV_R2_valid),stats.sem(CV_R2_valid))
print 'CV MSE (mean, std_err): ' + '{:04.2f}, {:04.2f}'.format(np.mean(CV_MSE_valid),stats.sem(CV_MSE_valid))

Loading previously saved model: 
CV R2 (mean, std_err): 0.23, 0.02
CV MSE (mean, std_err): 59.45, 3.04


In [32]:
# Evaluate the test set based on most frequnt hyper-params
if train_clf:
    #test_clf = LinearRegression()  
    #test_clf = Lasso(alpha=0.05)    
    test_clf = SVR(kernel='linear',C=1)
    #test_clf = RandomForestRegressor(n_estimators=500,min_samples_split=1)
    test_clf.fit(cv_data['X'],cv_data['y'])

    save_model = True
    save_model_path = CV_model_dir + model_choice + '_' + clinical_scale[0]
    if save_model:
        classifier_model_and_stats = {'best_clf':test_clf, 'CV_R2':CV_R2_valid, 'CV_MSE':CV_MSE_valid}
        save_classifier(classifier_model_and_stats,save_model_path)

pearson_r_train = stats.pearsonr(test_clf.predict(cv_data['X']),cv_data['y'])
R2_train = test_clf.score(cv_data['X'],cv_data['y'])
MSE_train = mse(test_clf.predict(cv_data['X']),cv_data['y'])

pearson_r_test = stats.pearsonr(test_clf.predict(test_data['X']),test_data['y'])
R2_test = test_clf.score(test_data['X'],test_data['y'])
MSE_test = mse(test_clf.predict(test_data['X']),test_data['y'])

print "train r: " + str(pearson_r_train), "test r: " + str(pearson_r_test)
print "train R2 score: " + str(R2_train), "train MSE: " + str(MSE_train)
print "test R2 score: " + str(R2_test), "test MSE: " + str(MSE_test)

ValueError: Number of features of the model must  match the input. Model n_features is 76 and  input n_features is 22020 

In [39]:
#R2_list = []
#MSE_list = []
#R2_list.append(CV_R2_valid)
#MSE_list.append(CV_MSE_valid)
boxplot_config_R2 = collections.defaultdict(list)
boxplot_config_MSE = collections.defaultdict(list)
method_labels = ['LR_L1', 'SVM', 'RFR']

for i in np.arange(3):
    boxplot_config_R2[method_labels[i]].append(R2_list[i])
    boxplot_config_MSE[method_labels[i]].append(MSE_list[i])

In [54]:
from matplotlib.artist import setp
plt.figure()
font_small = 12
font_med = 16
font_large = 24
plt.style.use('ggplot')

plt.subplot(2,1,1)
df_array = pd.DataFrame(dict([ (k,pd.Series(v[0])) for k,v in boxplot_config_R2.iteritems() ]))
bplot = df_array.boxplot(column=method_labels, fontsize=font_large)
#plt.xlabel('Method',fontsize=font_large)
plt.ylabel('R2',fontsize=font_large)
plt.xticks(fontsize=font_med)
plt.yticks(fontsize=font_small)
setp(bplot['boxes'], linewidth=2)
setp(bplot['whiskers'], linewidth=2)
setp(bplot['fliers'], linewidth=2)
setp(bplot['medians'], linewidth=2)

plt.subplot(2,1,2)
df_array = pd.DataFrame(dict([ (k,pd.Series(v[0])) for k,v in boxplot_config_MSE.iteritems() ]))
bplot = df_array.boxplot(column=method_labels, fontsize=font_large)
#plt.xlabel('Method',fontsize=font_large)
plt.ylabel('MSE',fontsize=font_large)

#plt.title('Number of Atlases',fontsize=font_large)
#plt.ylim([.5,1.0])
plt.xticks(fontsize=font_med)
plt.yticks(fontsize=font_small)
setp(bplot['boxes'], linewidth=2)
setp(bplot['whiskers'], linewidth=2)
setp(bplot['fliers'], linewidth=2)
setp(bplot['medians'], linewidth=2)
    
plt.show()

/home/nikhil/.conda/envs/snowflakes/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: 
The default value for 'return_type' will change to 'axes' in a future release.
 To use the future behavior now, set return_type='axes'.
 To keep the previous behavior and silence this warning, set return_type='dict'.
/home/nikhil/.conda/envs/snowflakes/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: 
The default value for 'return_type' will change to 'axes' in a future release.
 To use the future behavior now, set return_type='axes'.
 To keep the previous behavior and silence this warning, set return_type='dict'.


In [8]:
# Compute scores and MSEs

y_cv_pred = test_clf.predict(cv_data['X'])
y_test_pred = test_clf.predict(test_data['X'])

x_data_array = [cv_data['y'],test_data['y']]
y_data_array = [y_cv_pred,y_test_pred]
lable_array = ['CV train performance','test performance']

# only test perf
#x_data_array = [test_data['y']]
#y_data_array = [y_test_pred]
#lable_array = ['test performance']

plt.figure()
font_small = 8
font_med = 16
font_large = 24
no_of_plots = len(lable_array)
plt.style.use('ggplot')
plt_col = no_of_plots
plt_row = 1

for i in np.arange(no_of_plots):
    x = x_data_array[i]
    y = y_data_array[i]

    plt.subplot(plt_row,plt_col,i+1)
    plt.scatter(x, y, c='crimson', label=lable_array[i],s=40)
    fit = np.polyfit(x,y,1)
    fit_fn = np.poly1d(fit) 
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    if p_value < 0.0001:
        p_value_sig = '<0.0001'
    else:
        p_value_sig = str(p_value)
        
    label_str = 'r-value: {:04.2f}'.format(r_value) + '\n' + 'p-value: ' + p_value_sig + '\n' + 'std_err: {:04.2f}'.format(std_err) 
    # fit_fn is now a function which takes in x and returns an estimate for y
    plt.plot(x, fit_fn(x),linewidth=3, label=label_str)
    plt.title(model_choice,fontsize=font_large)
    plt.xlabel('Actual Score',fontsize=font_large)
    plt.ylabel('Predicted Score',fontsize=font_large)            
    plt.legend(fontsize=font_med,loc=2)

if feat_imp:
    plt.figure()
    #plt.subplot(plt_row,plt_col,4)
    x_pos = np.arange(len(feature_cols))
    
    if model_choice == 'RFR':
        feature_wts = test_clf.feature_importances_
    elif model_choice == 'LR_L1':
        feature_wts = np.squeeze(test_clf.coef_)
    else: 
        print 'no feature_wt vector found'
        
    sorted_feat_idx = np.argsort(np.abs(feature_wts))[::-1]        
    plt.bar(x_pos,feature_wts[sorted_feat_idx],color='crimson')
    plt.ylabel('Feature Importance',fontsize=font_large)
    #Sort the feature name list as well 
    sorted_feature_cols = []
    for i,sort_idx in enumerate(sorted_feat_idx):
        sorted_feature_cols.append(feature_cols[sort_idx])

    plt.xticks(x_pos,sorted_feature_cols,rotation=70,fontsize=font_small)
    
plt.show()

NotFittedError: This Lasso instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
# Additional Scripts 
# Concat of Train + Valid (to generate multi-folds)
t_data = pd.read_pickle(train_file)
v_data = pd.read_pickle(valid_file)
frames = [t_data, v_data]
result = pd.concat(frames)
result.to_pickle("/projects/nikhil/ADNI_prediction/input_datasets/cli_ct_seg_fused_train_plus_val.pkl")

# Generatng K-Folds
sampx = 100 #Train + Valid samples
foldx = 10   
kf = KFold(sampx, n_folds=foldx,shuffle=True)

#for train, test in kf:
#    print("%s %s" % (train, test))